<a href="https://colab.research.google.com/github/Tiagogavinhos/Music-Recommendation-System---Spotify/blob/main/Music_Recommendation_System_Spotify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
################################################################################
# Import libraries on Python
################################################################################

import string
import numpy as np
from sklearn.neighbors import NearestNeighbors
import nltk
from nltk.corpus import stopwords
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')
import kagglehub
import pandas as pd
import os
import re
from sklearn import metrics

In [46]:
################################################################################
################################################################################

# Dataset below was collected on Kaggle - Website is below:
# https://www.kaggle.com/datasets/notshrirang/spotify-million-song-dataset/data
# The purpose of this project is to recomend musics based on the previous musics
# listened

################################################################################
################################################################################

# 1. Baixa o dataset (o path será algo como /root/.cache/kagglehub/datasets/...)
path = kagglehub.dataset_download("notshrirang/spotify-million-song-dataset")

print("O arquivo foi baixado para:", path)

# 2. Listar os arquivos dentro do path para encontrar o nome correto do CSV
arquivos = os.listdir(path)
print("Arquivos encontrados:", arquivos)

# 3. Concatenar o caminho da pasta com o nome do arquivo CSV
# Vamos pegar o primeiro arquivo .csv que encontrarmos na pasta
csv_file = [f for f in arquivos if f.endswith('.csv')][0]
full_path = os.path.join(path, csv_file)

# 4. Agora sim, carregar usando o caminho completo
df = pd.read_csv(full_path)

# Visualizar as primeiras linhas
display(df.head())

Using Colab cache for faster access to the 'spotify-million-song-dataset' dataset.
O arquivo foi baixado para: /kaggle/input/spotify-million-song-dataset
Arquivos encontrados: ['spotify_millsongdata.csv']


,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [47]:
################################################################################
# Let's make some exploratory analyze
################################################################################

print(df.info())

################################################################################
# Missing innformation
################################################################################

print("\n")
print("Checking missing info:")
print(df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57650 entries, 0 to 57649
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  57650 non-null  object
 1   song    57650 non-null  object
 2   link    57650 non-null  object
 3   text    57650 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB
None


Checking missing info:
artist    0
song      0
link      0
text      0
dtype: int64


In [48]:
################################################################################
# Steps:
# A) It is not necessary the link on dateset. It will be removed
# B) Rename 'Text' per 'Lyrics'
# C) Remove duplicated song on dataset if has it
# D) Reset the index
################################################################################

# Check if 'link' column exists before dropping it

if 'link' in df.columns:
    df.drop(["link"], axis = 1, inplace = True)
df.rename(columns = {"text":"lyrics"}, inplace = True)
df.drop_duplicates(subset="song", inplace=True)
df.reset_index(drop = True, inplace = True)


print(df.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44824 entries, 0 to 44823
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  44824 non-null  object
 1   song    44824 non-null  object
 2   lyrics  44824 non-null  object
dtypes: object(3)
memory usage: 1.0+ MB
None


In [49]:
################################################################################
# Steps:
# E) Creating a new column - Concatenate
################################################################################

df["text_together"] = df["artist"]+ " "+df["song"]+" "+df["lyrics"]
print(df.head())
print("\n")
print(df.info())

  artist                   song  \
0   ABBA  Ahe's My Kind Of Girl   
1   ABBA       Andante, Andante   
2   ABBA         As Good As New   
3   ABBA                   Bang   
4   ABBA       Bang-A-Boomerang   

                                              lyrics  \
0  Look at her face, it's a wonderful face  \r\nA...   
1  Take it easy with me, please  \r\nTouch me gen...   
2  I'll never know why I had to go  \r\nWhy I had...   
3  Making somebody happy is a question of give an...   
4  Making somebody happy is a question of give an...   

                                       text_together  
0  ABBA Ahe's My Kind Of Girl Look at her face, i...  
1  ABBA Andante, Andante Take it easy with me, pl...  
2  ABBA As Good As New I'll never know why I had ...  
3  ABBA Bang Making somebody happy is a question ...  
4  ABBA Bang-A-Boomerang Making somebody happy is...  


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44824 entries, 0 to 44823
Data columns (total 4 columns):
 #   Column 

In [50]:
################################################################################
# Steps:
# F) Cleaning text - Removing link words, Upper Letters and so on
################################################################################

import re
import nltk
from nltk.corpus import stopwords

# Download the stopwords corpus if not already downloaded
nltk.download('stopwords')

def cleaning(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    text = text.lower()
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

df["text_together"] = df["text_together"].apply(func = cleaning)

print("\n ********** Checking if working ********** \n")
print(df.text_together[0])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



 ********** Checking if working ********** 

abba ahe kind girl look face wonderful face means something special look way smiles sees lucky one fellow kind girl makes feel fine could ever believe could mine kind girl without blue ever leaves could could go walk park holds squeezes hand go walking hours talking things plan kind girl makes feel fine could ever believe could mine kind girl without blue ever leaves could could


In [51]:
################################################################################
# Steps:
# G) Vectorization - Text into numerical vectors
# H) Evaluate the size
################################################################################

Text_Classification = TfidfVectorizer(max_features = 10000)
Text_Matrix = Text_Classification.fit_transform(df["text_together"])
print("Size of Text_Matrix:",Text_Matrix.shape)

# PS: To limit the size the script is:
# Text_Classification = TfidfVectorizer(max_features = 10000)
# As the processing is not long, it will be not input any limitation on the process


Size of Text_Matrix: (44824, 10000)


In [52]:
################################################################################
# Steps:
# I) Cosine similarity - Check the closeness of two vectors
################################################################################

model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(Text_Matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [53]:
################################################################################
# Steps:
# J) Finding 5 (five) similar music based on the first music
################################################################################

# 5. Exemplo de Busca: Encontrar 5 músicas similares à primeira da amostra
distances, indices = model_knn.kneighbors(Text_Matrix[0], n_neighbors=6)

for i in range(1, len(distances.flatten())):
    print(f"Suggestion {i}: {df.iloc[indices.flatten()[i]]['song']} - {df.iloc[indices.flatten()[i]]['artist']}")

Suggestion 1: The Kind Of Girl I Could Love - The Monkees
Suggestion 2: What Kind Of Girl - Air Supply
Suggestion 3: Could I - Bread
Suggestion 4: That Boy - Lloyd Cole
Suggestion 5: Don't Be Kind - John Denver
